# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
X. Zhang  ->  X. Zhang  |  ['X. Zhang']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
F. Xu  ->  F. Xu  |  ['F. Xu']


J. Li  ->  J. Li  |  ['J. Li']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
N. Bachmann  ->  N. Bachmann  |  ['N. Bachmann']
H.-W. Rix  ->  H.-W. Rix  |  ['H.-W. Rix']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
J. Müller-Horn  ->  J. Müller-Horn  |  ['J. Müller-Horn']
Arxiv has 107 new papers today
          9 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/9 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2511.11920


extracting tarball to tmp_2511.11920...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2511.11920/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: '10_appendixB' from 'tmp_2511.11920/10_appendixB.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: '09_appendixA' from 'tmp_2511.11920/09_appendixA.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/

...

/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\hsize]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[angle=-90,width=3cm]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_d

/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[bb=10 20 100 300,clip]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


Found 170 bibliographic references in tmp_2511.13483/aanda.bbl.
Retrieving document from  https://arxiv.org/e-print/2511.13692


extracting tarball to tmp_2511.13692...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.11920-b31b1b.svg)](https://arxiv.org/abs/2511.11920) | **PAH Marks the Spot: Digging for Buried Clusters in Nearby Star-forming Galaxies**  |
|| G. B. Graham, et al. -- incl., <mark>E. Schinnerer</mark> |
|*Appeared on*| *2025-11-18*|
|*Comments*| *27 pages, 14 figures. To be published in The Astronomical Journal*|
|**Abstract**|            The joint capabilities of the Hubble Space Telescope (HST) and JWST allow for an unparalleled look at the early lives of star clusters at near- and mid-infrared wavelengths. We present here a multiband analysis of embedded young stellar clusters in 11 nearby, star-forming galaxies, using the PHANGS-JWST and PHANGS-HST datasets. We use the Zooniverse citizen science platform to conduct an initial by-eye search for embedded clusters in near-UV/optical/near-infrared images that trace stellar continuum emission, the Paschen$\alpha$ and H$\alpha$ recombination lines, and the 3.3 $\mu$m polycyclic aromatic hydrocarbon feature and its underlying continuum. With this approach, we identify 292 embedded cluster candidates for which we characterize their ages, masses, and levels of line-of-sight extinction by comparing the photometric data to predictions from stellar population models. The embedded cluster candidates have a median age of 4.5 Myr and an average line-of-sight extinction $\left< A_V \right> = 6.0$ mag. We determine lower limits on source stellar masses, resulting in a median stellar mass of $10^3$ $M_{\odot}$. We use this sample of embedded cluster candidates to train multiple convolutional neural network models to carry out deep transfer learning-based searches for embedded clusters. With the aim of optimizing models for future catalog production, we compare results for four variations of training data using two neural networks. Confusion matrices for all eight model configurations, as well as inter-model identification trends, are presented. With refinement of the training sample, we determine that optimized models could serve as a pathway for future embedded cluster identification beyond our 11 galaxy sample.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.11943-b31b1b.svg)](https://arxiv.org/abs/2511.11943) | **Euclid Quick Data Release (Q1): Identification of massive galaxy candidates at the end of the Epoch of Reionisation**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-11-18*|
|*Comments*| *13 pages, including 6 figures and 3 tables; submitted to A&A*|
|**Abstract**|            Probing the presence and properties of massive galaxies at high redshift is one of the most critical tests for galaxy formation models. In this work, we search for galaxies with stellar masses M* > 10^10.25 Msun at z in [5,7], i.e., towards the end of the Epoch of Reionisation, over a total of ~23 deg^2 in two of the Euclid Quick Data Release (Q1) fields: the Euclid Deep Field North and Fornax (EDF-N and EDF-F). In addition to the Euclid photometry, we incorporate Spitzer Infrared Camera (IRAC) and ground-based optical data to perform spectral energy distribution (SED) fitting, obtaining photometric redshifts and derived physical parameters. After applying rigorous selection criteria, we identify a conservative sample of 145 candidate massive galaxies with M* > 10^10.25 Msun at z in [5,7], including 5 objects with M* > 10^11 Msun. This makes for a surface density of about 6.3 deg^-2 at z in [5,7], which should be considered a lower limit because of the current depth of the Euclid data (H_E < 24, 5 sigma in Q1). We find that the inferred stellar masses are consistent with galaxy formation models with standard star-formation efficiencies. These massive galaxies have colour excess E(B-V) values up to 0.75, indicating significant dust attenuation in some of them. In addition, half of the massive galaxies have best-fit ages comparable to the age of the Universe at those redshifts, which suggests that their progenitors were formed very early in cosmic time. About 78% of the massive galaxies lie on the star-forming main sequence (MS) in the SFR-M* plane, ~12% are found in the starburst region, and 10% in the transition zone between the MS and starbursts. We find no significant evidence for outshining or AGN contamination that could account for the elevated specific star-formation rates (sSFR) observed in the ~12% of galaxies classified as starbursts.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.13483-b31b1b.svg)](https://arxiv.org/abs/2511.13483) | **The bulk metal content of WASP-80 b from joint interior-atmosphere retrievals: Breaking degeneracies and exploring biases with panchromatic spectra**  |
|| L. Acuña-Aguirre, <mark>L. Kreidberg</mark>, <mark>P. Mollière</mark>, <mark>N. Bachmann</mark> |
|*Appeared on*| *2025-11-18*|
|*Comments*| *19 pages, 6 figures plus appendix. Under review in Astronomy & Astrophysics. Version after first referee report*|
|**Abstract**|            WASP-80 b is an unusually low-density exoplanet in tension with the metal-rich composition expected for a planet of its mass. We aim to derive precise constraints on WASP-80 b's bulk metal mass fraction, atmospheric composition, and thermal structure. We conducted a suite of retrievals using three approaches: traditional interior-only, atmosphere-only, and joint interior-atmosphere retrievals. We coupled the open-source models GASTLI and petitRADTRANS, which describe planetary structure and thermal evolution, and atmospheric chemistry and clouds, respectively. Our retrievals combine mass and age with panchromatic spectra from JWST and HST in both transmission (0.5-4 $\mu$m) and emission (1-12 $\mu$m) as observational constraints. We identify two fiducial scenarios. In the first, WASP-80 b has an internal temperature consistent with its age in the absence of external heating sources, and its atmosphere is in chemical equilibrium, with an atmospheric metallicity M/H = 2.75$^{+0.88}_{-0.56}$x solar, a bulk metal mass fraction $Z_{planet}=0.12\pm0.02$, and a core mass $M_{core}=3.49^{+3.49}_{-1.59} \ M_{\oplus}$. In the second scenario, WASP-80 b may be inflated by an additional heat source - possibly induced by magnetic fields - with an atmospheric metallicity M/H = 10.00$^{+8.20}_{-4.75}$x solar, $Z_{planet}=0.28\pm0.11$, and $M_{core}=31.8^{+21.3}_{-17.5} \ M_{\oplus}$. The super-solar M/H and sub-solar C/O ratios in both scenarios suggest late pebble or planetesimal accretion, while additional heating is required to reconcile the data with the more massive core predicted by the core accretion paradigm. In general, joint retrievals are inherently affected by a degeneracy between atmospheric chemistry and internal structure. Together with flexible cloud treatment and an unweighted likelihood, this leads to larger uncertainties in bulk and atmospheric compositions than previously claimed.         |

## Failed papers

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.12017-b31b1b.svg)](https://arxiv.org/abs/2511.12017) | **Gravitational wave standard sirens from GWTC-3 combined with DESI DR2 and DESY5: A late-universe probe of the Hubble constant and dark energy**  |
|| J.-Y. Song, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-11-18*|
|*Comments*| *10 pages, 3 figures*|
|**Abstract**|            Recently, the combination of the Dark Energy Spectroscopic Instrument (DESI) Data Release 2 (DR2) baryon acoustic oscillation (BAO) data and the Planck cosmic microwave background (CMB) measurements has shown a $\sim$3$\sigma$ preference for a dynamical dark energy model with a phantom-crossing behavior. However, such a phantom-crossing dark energy evolution further exacerbates the already severe Hubble tension in the $\Lambda$CDM model. Moreover, there exists a $\sim2\sigma$ tension between the DESI DR2 BAO and CMB datasets. Therefore, it is essential to measure the Hubble constant and dark-energy equation-of-state (EoS) parameters using only late-universe observations. In this work, we investigate a novel late-universe data combination: gravitational-wave (GW) standard sirens, BAO, and Type Ia supernovae (SNe Ia). This combination provides a fully distance-ladder- and CMB-independent determination of the Hubble constant and the dark-energy EoS. Using 47 GW standard sirens from the third Gravitational-Wave Transient Catalog, the DESI DR2 BAO data, and DESY5 SNe Ia data, in the $w_0w_a$CDM model, we obtain $H_0=74.8^{+6.3}_{-8.9}$ km s$^{-1}$ Mpc$^{-1}$, $\Omega_{\rm m}=0.320^{+0.015}_{-0.012}$, $w_0=-0.775^{+0.072}_{-0.074}$, and $w_a=-0.80\pm0.47$, indicating a mild phantom-crossing behavior within the $1\sigma$ credible interval with an $H_0$ value consistent with the distance ladder measurements. Our analysis demonstrates the power of GW standard sirens in breaking parameter degeneracies, and this novel data combination provides joint constraints on the Hubble constant and the dark-energy EoS parameters.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.12418-b31b1b.svg)](https://arxiv.org/abs/2511.12418) | **Examining Turbulence in Galactic Molecular Clouds - II: Continuity of Turbulence Cascading in a Portion of the Local Arm**  |
|| Y. Ma, et al. -- incl., <mark>M. Zhang</mark> |
|*Appeared on*| *2025-11-18*|
|*Comments*| *12 pages and 7 Figures*|
|**Abstract**|            We use $^{12}$CO (J=1-0) MWISP data to study turbulence in a segment of the Local Arm. Velocity slices at different kinematic distances show similar spatial power spectra (SPSs) and structure functions (SFs), demonstrating that the entire region forms a single turbulent field with a cascade extending from $\sim 400$ pc to sub-parsec scales. The SPS slopes of both the intensity and velocity fields exhibit a systematic scale dependence that approaches the values expected from turbulence models. Cloud-to-cloud VSFs follow similar trends to the pixel-by-pixel VSFs in the extended self-similarity (ESS) scaling, indicating that velocity differences among clouds arise from large-scale turbulent motions. Velocity- and intensity-increment maps reveal filamentary, intermittent structures. The PDFs of the velocity increments display strong non-Gaussianity and are well fitted by the normal inverse gaussian (NIG) distribution, whereas the intensity increments show much weaker tails. A simple energetic estimate suggests that Galactic differential rotation is able to supply the large-scale shear required to maintain the observed turbulence.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.12477-b31b1b.svg)](https://arxiv.org/abs/2511.12477) | **Detection of disk-jet co-precession in a tidal disruption event**  |
|| <mark>Y. Wang</mark>, et al. |
|*Appeared on*| *2025-11-18*|
|*Comments*| *76 pages, 13 figures (initial submission; in press at Science Advances)*|
|**Abstract**|            Theories and simulations predict that intense spacetime curvature near black holes bends the trajectories of light and matter, driving disk and jet precession under relativistic torques. However, direct observational evidence of disk-jet co-precession remains elusive. Here, we report the most compelling case to date: a tidal disruption event (TDE) exhibiting unprecedented 19.6-day quasi-periodic variations in both X-rays and radio, with X-ray amplitudes exceeding an order of magnitude. The nearly synchronized X-ray and radio variations suggest a shared mechanism regulating the emission regions. We demonstrate that a disk-jet Lense-Thirring precession model successfully reproduces these variations while requiring a low-spin black hole. This study uncovers previously uncharted short-term radio variability in TDEs, highlights the transformative potential of high-cadence radio monitoring, and offers profound insights into disk-jet physics.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.12481-b31b1b.svg)](https://arxiv.org/abs/2511.12481) | **Mock Observations for the CSST Mission: Multi-Channel Imager--Instrument Simulation**  |
|| Z.-J. Yan, et al. -- incl., <mark>F. Xu</mark> |
|*Appeared on*| *2025-11-18*|
|*Comments*| *28 pages, 21 figures, accepted by RAA*|
|**Abstract**|            The Chinese Space Station Survey Telescope (CSST), a two-meter aperture astronomical space telescope under China's manned space program, is equipped with multiple back-end scientific instruments. As an astronomical precision measurement module of the CSST, the Multi-Channel Imager (MCI) can cover a wide wavelength range from ultraviolet to near-infrared with three-color simultaneous high-precision photometry and imaging, which meets the scientific requirements for various fields. The diverse scientific objectives of MCI require not only a robust airborne platform, advanced optical systems, and observing facilities but also comprehensive software support for scientific operations and research. To this end, it is essential to develop realistic observational simulation software to thoroughly evaluate the MCI data stream and provide calibration tools for future scientific investigations. The MCI instrument simulation software will serve as a foundation for the development of the MCI data processing pipeline and will facilitate improvements in both hardware and software, as well as in the observational operation strategy, in alignment with the mission's scientific goals. In conclusion, we present a comprehensive overview of the MCI instrument simulation and some corresponding performances of the MCI data processing pipeline.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2511.13094-b31b1b.svg)](https://arxiv.org/abs/2511.13094) | **Wide-Field X-ray Polarimetry for High Energy Astronomical Transients: First results of the pathfinder CXPD Cubesat Mission**  |
|| H.-B. Liu, et al. -- incl., <mark>J. Li</mark> |
|*Appeared on*| *2025-11-18*|
|*Comments*| **|
|**Abstract**|            The Low Energy Polarization Detector (LPD) is a key component of the next-generation large-scale Gamma-Ray Burst polarimeter, POLAR-2. It is designed for polarization observations of transient sources in the soft X-ray energy range with a wide field of view (FOV). To validate the key technologies required for wide-FOV X-ray polarization measurements, the Cosmic X-ray Polarization Detector (CXPD) CubeSat was developed as a prototype for the LPD. The CXPD is equipped with two Gas Microchannel Plate Pixel Detectors (GMPDs) that measure X-ray polarization via the photoelectric effect, where ejected photoelectrons produce ionization tracks in the gas which are imaged to reconstruct their emission directions. Laboratory calibrations of the modulation factor and energy spectra were successfully performed using linear polarized X-ray sources at 2.98 keV, 4.51 keV, 6.40 keV, and 8.05 keV. Since its launch in June 2023, the CXPD has successfully completed critical in-orbit technology verification. It has also performed polarization observations of two bright X-ray sources Sco X-1 and the transient Swift J1727.8-1613 yielding constraints on their polarization degrees and angles. Notably, this was the first time that an anti-coincidence detector had been implemented in an X-ray polarimeter, enabling in-orbit verification of the charged-particle background rejection algorithm. These results demonstrate the feasibility of wide-field soft X-ray polarization measurements and provide essential guidance for the development of the LPD for the POLAR-2 mission, thereby advancing the frontier of X-ray polarization astronomy.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['', '', '', '', '', '']
file not found 
file not found 
file not found 
file not found 
file not found 
file not found 
exported in  _build/html/2511.11920.md
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
    + _build/html/
found figures ['tmp_2511.11943/./EDFN_EDFF_lephare_V5.png', 'tmp_2511.11943/./mass_distrib_panel.png', 'tmp_2511.11943/./size_mass.png']
copying  tmp_2511.11943/./EDFN_EDFF_lephare_V5.png to _build/html/
copying  tmp_2511.11943/./mass_distrib_panel.png to _build/html/
copying  tmp_2511.11943/./size_mass.png to _build/html/
exported in  _build/html/2511.11943.md
    + _build/html/tmp_2511.11943/./EDFN_EDFF_lephare_V5.png
    + _build/html/tmp_2511.11943/./mass_distrib_panel.png
    + _build/html/tmp_2511.11943/./size_mass.png
found figures ['tmp_2511.13483/./Figures/PTprofile_PAPERPLOT_3panels.png', 'tmp_2511.13483/./Figures/CMF_Zplanet_PAPER_wiser_2panels.png', '', '']
copying  tmp_2511.13483/./Figures/

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\comment}[2]{{\color{gray} #1:~ #2}}$
$\newcommand{\sbullet}{\mathbin{\vcenter{\hbox{\scalebox{0.7}{\bullet}}}}}$
$\newcommand{\sbsc}[1]{_\mathrm{#1}}$
$\newcommand{\spsc}[1]{^\mathrm{#1}}$
$\newcommand{\brkt}[1]{\left<#1\right>}$
$\newcommand{\hal}{H\alpha}$
$\newcommand{\hb}{H\beta}$
$\newcommand{\paal}{Pa\alpha}$
$\newcommand{\paab}{Pa\beta}$
$\newcommand{\HI}{\ion{H}{i}}$
$\newcommand{\HII}{\ion{H}{ii}}$
$\newcommand{\SII}{[\ion{S}{ii}]}$
$\newcommand{\SIII}{[\ion{S}{iii}]}$
$\newcommand{\NII}{[\ion{N}{ii}]}$
$\newcommand{\OIII}{[\ion{O}{iii}]}$
$\newcommand{\Htwo}{\mbox{\mathrm{H}_2}}$
$\newcommand{\CO}[2]{\mbox{\mathrm{CO} (#1\text{--}#2)}}$
$\newcommand{\Spitzer}{\it Spitzer}$
$\newcommand{\IRAS}{\it IRAS}$
$\newcommand{\twoMASS}{\it twoMASS}$
$\newcommand{\GALEX}{\it GALEX}$
$\newcommand{\jwst}{JWST}$
$\newcommand{\hst}{HST}$
$\newcommand{\m}{\mum}$
$\newcommand{\Ks}{\it K_{\rm s}}$
$\newcommand{\SDSS}{\it SDSS}$
$\newcommand{\Mstar}{M_\star}$
$\newcommand{\Msun}{\rm{M_\odot}}$
$\newcommand{\UWyoming}{\affiliation{Department of Physics and Astronomy, University of Wyoming, Laramie, WY 82071, USA}}$
$\newcommand{\STScI}{\affiliation{Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\NOIRLab}{\affiliation{International Gemini Observatory/NSF NOIRLab, 950 N. Cherry Avenue, Tucson, AZ 85719, USA}}$
$\newcommand{\UToledo}{\affiliation{Ritter Astrophysical Research Center, University of Toledo, Toledo, OH 43606, USA}}$
$\newcommand{\JHU}{\affiliation{Department of Physics and Astronomy, The Johns Hopkins University, Baltimore, MD 21218 USA}}$
$\newcommand{\OSU}{\affiliation{Department of Astronomy, The Ohio State University, 140 West 18th Ave., Columbus, OH 43210, USA}}$
$\newcommand{\MPIA}{\affiliation{Max Planck Institut für Astronomie, Königstuhl 17, 69117 Heidelberg, Germany}}$
$\newcommand{\UCSD}{\affiliation{Department of Astronomy \& Astrophysics, University of California San Diego, 9500 Gilman Drive, La Jolla, CA 92093, USA}}$
$\newcommand{\ANU}{\affiliation{Research School of Astronomy and Astrophysics, Australian National University, Canberra, ACT 2611, Australia}}$
$\newcommand{\nrao}{\affiliation{National Radio Astronomy Observatory, 520 Edgemont Road, Charlottesville, VA 22903, USA}}$
$\newcommand{\MPE}{\affiliation{Max-Planck-Institut für Extraterrestrische Physik (MPE), Giessenbachstr. 1, D-85748 Garching, Germany}}$
$\newcommand{\Oxford}{\affil{Sub-department of Astrophysics, Department of Physics, University of Oxford, Keble Road, Oxford OX1 3RH, UK}}$
$\newcommand{\STScIESA}{\affiliation{AURA for the European Space Agency (ESA), Space Telescope Science Institute, 3700 San Martin Drive, Baltimore, MD 21218, USA}}$
$\newcommand{\UConn}{\affiliation{Department of Physics, University of Connecticut, Storrs, CT 06269, USA}}$
$\newcommand{\ESO}{\affiliation{European Southern Observatory, Karl-Schwarzschild Stra{\ss}e 2, D-85748 Garching bei München, Germany}}$
$\newcommand{\Plata}{\affiliation{Instituto de Astrofísica de La Plata, CONICET--UNLP, Paseo del Bosque S/N, B1900FWA La Plata, Argentina}}$
$\newcommand{\BrynMawr}{\affiliation{Department of Physics and Astronomy, Bryn Mawr College, Bryn Mawr, PA 19010, USA}}$
$\newcommand{\Virginia}{\affiliation{Department of Astronomy, University of Virginia, Charlottesville, VA 22904, USA}}$
$\newcommand{\Utah}{\affiliation{Department of Physics and Astronomy, University of Utah, Salt Lake City, UT 84112, USA}}$
$\newcommand{\Rice}{\affiliation{Department of Physics and Astronomy, Rice University, Houston, TX 77005, USA}}$
$\newcommand{\CPP}{\affiliation{Department of Physics and Astronomy, California State Polytechnic University Pomona, Pomona, CA 91768, USA}}$
$\newcommand{\Lycoming}{\affiliation{Department of Physics and Astronomy, Lycoming College, Williamsport, PA, 17701, USA}}$
$\newcommand{\CPH}{\affiliation{Department of Physics and Astronomy, California State Polytechnic University Humboldt, Arcata, CA 95521, USA$
$}}$
$\newcommand{\kipac}{\affiliation{Kavli Institute for Particle Astrophysics \& Cosmology (KIPAC), Stanford University, CA 94305, USA}}$
$\newcommand{\Whitman}{\affiliation{Whitman College, 345 Boyer Avenue, Walla Walla, WA 99362, USA}}$
$\newcommand{\UniCA}{\affiliation{Université Côte d'Azur, Observatoire de la Côte d'Azur, CNRS, Laboratoire Lagrange, 06000, Nice, France}}$
$\newcommand{\UOA}{\affiliation{Department of Physics, University of Arkansas, 226 Physics Building, 825 West Dickson Street, Fayetteville, AR 72701, USA}}$
$\newcommand{\Michigan}{\affiliation{Department of Astronomy, University of Michigan, Ann Arbor, MI 48109, USA}}$
$\newcommand{\UAlberta}{\affiliation{Department of Physics, University of Alberta, 4-183 CCIS, Edmonton, Alberta, T6G 2E1, Canada}}$
$\newcommand{\Ensenada}{\affiliation{Instituto de Astronomía, Universidad Nacional Autónoma de México, Unidad Académica en Ensenada, Km 103 Carr. Tijuana—Ensenada, Ensenada, B.C., C.P. 22860, México}}$
$\newcommand{\UNAM}{\affiliation{Instituto de Astronomía, Universidad Nacional Autónoma de México, Ap. 70-264, 04510 CDMX, Mexico}}$</div>



<div id="title">

# PAH Marks the Spot: Digging for Buried Clusters in Nearby Star-forming Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.11920-b31b1b.svg)](https://arxiv.org/abs/2511.11920)<mark>Appeared on: 2025-11-18</mark> -  _27 pages, 14 figures. To be published in The Astronomical Journal_

</div>
<div id="authors">

Gabrielle~B.~Graham, et al.

</div>
<div id="abstract">

**Abstract:** The joint capabilities of the Hubble Space Telescope (HST) and JWST allow for an unparalleled look at the early lives of star clusters at near- and mid-infrared wavelengths. We present here a multiband analysis of embedded young stellar clusters in 11 nearby, star-forming galaxies, using the PHANGS-JWST and PHANGS-HST datasets. We use the $_ Zooniverse_$ citizen science platform to conduct an initial by-eye search for embedded clusters in near-UV/optical/near-infrared images that trace stellar continuum emission, the Paschen $\alpha$ and H $\alpha$ recombination lines, and the 3.3 $\mu$ m polycyclic aromatic hydrocarbon feature and its underlying continuum. With this approach, we identify 292 embedded cluster candidates for which we characterize their ages, masses, and levels of line-of-sight extinction by comparing the photometric data to predictions from stellar population models.  The embedded cluster candidates have a median age of 4.5 Myr and an average line-of-sight extinction $\left< A_V \right> = 6.0$ mag. We determine lower limits on source stellar masses, resulting in a median stellar mass of $10^3$  $M_{\sun}$ . We use this sample of embedded cluster candidates to train multiple convolutional neural network models to carry out deep transfer learning-based searches for embedded clusters. With the aim of optimizing models for future catalog production, we compare results for four variations of training data using two neural networks. Confusion matrices for all eight model configurations, as well as inter-model identification trends, are presented. With refinement of the training sample, we determine that optimized models could serve as a pathway for future embedded cluster identification beyond our 11 galaxy sample.

</div>

<div id="div_fig1">

<img src="" alt="Fig13.1" width="50%"/><img src="" alt="Fig13.2" width="50%"/>

**Figure 13. -** Confusion matrices for all 8 model configurations, normalized to the number of objects tested in each `True Label' category (see Table \ref{tab:MLobjects} for number of objects in each class). The numbers below the percentages are the average and standard deviation of the model agreement in each quadrant. Left: Confusion matrices for models which include F150W and Pa$\alpha$ data in training. Right: Confusion matrices for models which exclude F150W and Pa$\alpha$ data in training. (*fig:C2CMs*)

</div>
<div id="div_fig2">

<img src="" alt="Fig14.1" width="25%"/><img src="" alt="Fig14.2" width="25%"/><img src="" alt="Fig14.3" width="25%"/><img src="" alt="Fig14.4" width="25%"/>

**Figure 14. -** Example ideal cluster candidates as seen in the _Zooniverse_ interface (left), a three-color NIRCam image (center), and a three-color HST image (right). From top to bottom, we show one candidate from NGC 1097, NGC 1559, NGC 2775, and NGC 2997. The four cutouts in the left panel are 2$\farcs$0 cutouts centered on the source using the same images as Figure \ref{fig:idCase}, with the cyan circle denoting the final F335M source center. The three-color images also display a 2$\farcs$0 cutout centered on the source. The individual bands are shown in the small boxes above the composite image. (*fig:Ideal_Sample*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.11920"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\lsim}{\raisebox{-0.13cm}{~\shortstack{< \\[-0.07cm] \sim}}~}$
$\newcommand{\gsim}{\raisebox{-0.13cm}{~\shortstack{> \\[-0.07cm] \sim}}~}$
$\newcommand{\orcid}[1]$
$\newcommand{\arraystretch}{1.1}$
$\newcommand{\arraystretch}{1.2}$</div>



<div id="title">

# Euclid Quick Data Release (Q1): Identification of massive galaxy candidates at the end of the Epoch of Reionisation

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.11943-b31b1b.svg)](https://arxiv.org/abs/2511.11943)<mark>Appeared on: 2025-11-18</mark> -  _13 pages, including 6 figures and 3 tables; submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** Probing the presence and properties of massive galaxies at high redshift is one of the most critical tests for galaxy formation models. In this work, we search for galaxies with stellar masses $\( M_* > 10^{10.25}   \rm{M_\odot} \)$ at $\( z \in [5,7]\)$ , i.e., towards the end of the Epoch of Reionisation, over a total of $\( \sim 23  \mathrm{deg}^2\)$ in two of the Euclid Quick Data Release (Q1) fields: the Euclid Deep Field North and Fornax (EDF-N and EDF-F). In addition to the _Euclid_ photometry, we incorporate _Spitzer_ Infrared Camera (IRAC) and ground-based optical data to perform spectral energy distribution (SED) fitting, obtaining photometric redshifts and derived physical parameters. After applying rigorous selection criteria,  we identify a conservative sample of 145 candidate massive galaxies with $\( M_* > 10^{10.25}   \rm{M_\odot} \)$ at $\( z \in [5,7] \)$ , including 5 objects with $\( M_* > 10^{11}   \rm{M_\odot} \)$ .  This makes for a surface density of about 6.3 deg $^{-2}$ at $z\in [5,7]$ , which should be considered a lower limit because of the current depth of the _Euclid_ data ( $\HE<24,   5\sigma$ in Q1). We find that the inferred stellar masses are consistent with galaxy formation models with standard star-formation efficiencies. These massive galaxies have colour excess $E(B-V)$ values up to 0.75, indicating significant dust attenuation in some of them. In addition, half of the massive galaxies have best-fit ages comparable to the age of the Universe at those redshifts, which suggests that their progenitors were formed very early in cosmic time. About 78 \% of the massive galaxies lie on the star-forming main sequence (MS) in the SFR– $\( M_* \)$ plane, $\sim$ 12 \% are found in the starburst region, and 10 \% in the transition zone between the MS and starbursts. We find no significant evidence for outshining or AGN contamination that could account for the elevated specific star-formation rates (sSFR) observed in the $\(\sim12\%\)$ of galaxies classified as starbursts.

</div>

<div id="div_fig1">

<img src="tmp_2511.11943/./EDFN_EDFF_lephare_V5.png" alt="Fig1" width="100%"/>

**Figure 1. -** Photometric redshifts that we derive in our work versus the spectroscopic redshifts from the literature available in EDF-N and EDF-F. For a more detailed description of the spectroscopic sample, we refer the reader to Q1-TP005. We report an outlier fraction of $9.6\%$ over $\sim 42 000$ galaxies in $z\in(0,6)$ at all magnitudes. We show the identity as a red continuous line, together with the shaded area delimiting catastrophic outliers, defined as $|z_{\rm phot}-z_{\rm spec}|/(1+z_{\rm spec})>0.15$. These cases are highlighted in red colour. (*fig:zphot_zspec*)

</div>
<div id="div_fig2">

<img src="tmp_2511.11943/./mass_distrib_panel.png" alt="Fig6" width="100%"/>

**Figure 6. -** *Left panel:* Stellar mass ($M_* $) versus photometric redshift. Our massive galaxy candidates are shown as red squares, overlaid on a gray hexagonal density plot representing the Q1 sample. The 5 candidates above $M_* >10^{11}  \rm{M_\odot}$ are shown with a black outline. For comparison, we include massive galaxies reported in the literature: orange triangles from chworowsky_evidence_2024, green hexagons from xiao_accelerated_2024(with black outlines highlighting the three most massive objects S1, S2, and S3) and a pink star from xiao_panoramic_2025.
    *Right panel:* Zoom-in of the left panel, focusing on our massive galaxy candidates shown as red squares. Confidence intervals from the Extreme Value Statistics (EVS) model  (lovell_extreme_2023) , computed for a survey area comparable to ours, are shown as blue shaded regions, by assuming a lognormal distribution of the star-formation efficiency (SFE). The gray shaded area at the top represents the absolute upper limit under the assumption of 100\% SFE.
     (*fig:hmf_panel*)

</div>
<div id="div_fig3">

<img src="tmp_2511.11943/./size_mass.png" alt="Fig2" width="100%"/>

**Figure 2. -** Size–stellar mass relation for VIS-detected galaxies in our sample (36 objects), showing the Sérsic effective radius from \IE modeling as a function of stellar mass. Our high-mass galaxies are represented by red squares. Observational relations from recent JWST-based studies are included as shaded regions: blue for ward_evolution_2024 and purple for yang_cosmos-web_2025. The minimum effective radius (\IE) at the average redshift of our sample is shown as a gray line. (*fig:size_mass*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.11943"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# The bulk metal content of WASP-80 b from joint interior–atmosphere retrievals: Breaking degeneracies and exploring biases with panchromatic spectra

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2511.13483-b31b1b.svg)](https://arxiv.org/abs/2511.13483)<mark>Appeared on: 2025-11-18</mark> -  _19 pages, 6 figures plus appendix. Under review in Astronomy & Astrophysics. Version after first referee report_

</div>
<div id="authors">

L. Acuña-Aguirre, <mark>L. Kreidberg</mark>, <mark>P. Mollière</mark>, <mark>N. Bachmann</mark>

</div>
<div id="abstract">

**Abstract:** The atmospheres of warm gas giants can be readily characterized through transmission and emission spectroscopy. WASP-80 b is one such exoplanet, with an unusually low density that is in tension with the metal-rich composition expected for a planet of this mass.   We aim to derive precise constraints on WASP-80 b’s bulk metal mass fraction, atmospheric composition, and thermal structure.  We conducted a suite of retrievals using three approaches: traditional interior-only, atmosphere-only, and joint interior–atmosphere retrievals. We coupled the open-source models GASTLI and petitRADTRANS, which describe planetary structure and thermal evolution, and atmospheric chemistry and clouds, respectively. Our retrievals combine mass and age with panchromatic spectra from JWST and HST in both transmission (0.5–4 $\mu$ m) and emission (1–12 $\mu$ m) as observational constraints.   We identify two fiducial scenarios. In the first, WASP-80 b has an internal temperature consistent with its age in the absence of external heating sources, and its atmosphere is in chemical equilibrium, with an atmospheric metallicity M/H = 2.75 $^{+0.88}_{-0.56}\times$ solar, a bulk metal mass fraction $Z_{\rm planet}=0.12\pm0.02$ , and a core mass $M_{\rm core} = 3.49^{+3.49}_{-1.59}  M_{\oplus}$ . In the second scenario, WASP-80 b may be inflated by an additional heat source -- possibly induced by magnetic fields -- with an atmospheric metallicity M/H = 10.00 $^{+8.20}_{-4.75}\times$ solar, $Z_{\rm planet}=0.28\pm0.11$ , and $M_{\rm core}=31.8^{+21.3}_{-17.5}  M_{\oplus}$ . The super-solar M/H and sub-solar C/O ratios in both scenarios suggest late pebble or planetesimal accretion, while additional heating is required to reconcile the data with the more massive core predicted by the core accretion paradigm. In general, joint retrievals are inherently affected by a degeneracy between atmospheric chemistry and internal structure. Together with flexible cloud treatment and an unweighted likelihood, this leads to larger uncertainties in bulk and atmospheric compositions than previously claimed.

</div>

<div id="div_fig1">

<img src="tmp_2511.13483/./Figures/PTprofile_PAPERPLOT_3panels.png" alt="Fig14" width="100%"/>

**Figure 14. -** Pressure-Temperature (PT) profiles of WASP-80 b as constrained by four of our joint retrievals. The shaded regions indicate the 1, 2, and 3$\sigma$ regions of the retrieval, while solid lines correspond to the mean. Black lines highlight self-consistent models from [Mollière and Bouwman (2017)](), [Acuña, et. al (2024)]() of a clear atmosphere with solar composition at $T_{\rm int}$ = 100 (dashed) and 500 K (dotted) for comparison. **Left**: The difference between JR3 and JR5 shows the effect of adding the transmission spectrum to JR3, which incorporates emission spectra plus the mass and radius. The transmission spectrum reduces significantly the uncertainties in thermal structure. **Center**: Comparing JR6 and JR5 allows us to study the effect of including the age to the retrieval. JR6 is the most-data complete of the joint retrievals, since it incorporates both transmission and emission, plus the mass and age. If the age is removed (JR5), the temperature in the deep atmosphere increases. **Right**: The comparison between JR5 and JR5$^{\ast}$ illustrates the effect of assuming equilibrium chemistry in the joint retrieval. If we relax the assumption of equilibrium chemistry in JR5 by changing to a free chemistry model (JR5$^{\ast}$), the temperature decreases in the deep interior. (*fig:ptprofile_joint*)

</div>
<div id="div_fig2">

<img src="tmp_2511.13483/./Figures/CMF_Zplanet_PAPER_wiser_2panels.png" alt="Fig8" width="100%"/>

**Figure 8. -** Top panel: Core and bulk metal mass fraction 1-$\sigma$ estimates obtained by our joint retrievals (JR1-JR6). Joint retrievals that take into account the age have triangle markers. Interior-only retrievals (R1-R3 and R5) are shown as colored boxes for comparison. Bottom panel: Atmospheric metallicity and C/O ratio estimates from our suite of joint retrievals. Spectra-only estimates are displayed as colored boxes for comparison. (*fig:joint_summary*)

</div>
<div id="div_fig3">

<img src="" alt="Fig5.1" width="50%"/><img src="" alt="Fig5.2" width="50%"/>

**Figure 5. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              List of nearby SNe used in this work.Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued. (*FigVibStab*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2511.13483"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

124  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

8  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

5  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
